## C. Increate the number of epochs (5 marks)

Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?

## Download and Clean Dataset

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's download the data and read it into a <em>pandas</em> dataframe.

In [2]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.

In [10]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

## Import Keras

#### Let's go ahead and import the Keras library

In [11]:
import keras

Using TensorFlow backend.
C:\Users\Home\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Home\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Home\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Home\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [12]:
from keras.models import Sequential
from keras.layers import Dense

## Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [13]:
# define regression model as my_model
def my_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Above my_model function has:
- One hidden layer of 10 nodes, and a ReLU activation function
- Use the adam optimizer and the mean squared error as the loss function.


Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_split helper function from Scikit-learn.

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=0)

## Train and Test the Network

Let's call the function now to create our model.

In [15]:
# build the model
model = my_model()

We will train the model for 100 epochs.

In [16]:
# fit the model
model.fit(X_train,y_train, epochs=100, verbose=1)


Epoch 1/100
721/721 [==============================] - 0s 161us/step - loss: 1584.0789
Epoch 2/100
721/721 [==============================] - 0s 22us/step - loss: 1569.6502
Epoch 3/100
721/721 [==============================] - 0s 43us/step - loss: 1555.8959
Epoch 4/100
721/721 [==============================] - 0s 52us/step - loss: 1542.6201
Epoch 5/100
721/721 [==============================] - 0s 22us/step - loss: 1529.8295
Epoch 6/100
721/721 [==============================] - 0s 22us/step - loss: 1516.9775
Epoch 7/100
721/721 [==============================] - 0s 43us/step - loss: 1504.0993
Epoch 8/100
721/721 [==============================] - 0s 31us/step - loss: 1490.8987
Epoch 9/100
721/721 [==============================] - 0s 43us/step - loss: 1477.2850
Epoch 10/100
721/721 [==============================] - 0s 43us/step - loss: 1462.9242
Epoch 11/100
721/721 [==============================] - 0s 43us/step - loss: 1447.5988
Epoch 12/100
721/721 [============================

Evaluate the model on test data

In [17]:
eval_model=model.evaluate(X_test,y_test)
y_pred=model.predict(X_test)

eval_model

309/309 [==============================] - 0s 101us/step


150.6537760515429

Compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

In [18]:
from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(y_test, y_pred) #mean squared error
mean = np.mean(MSE) #mean of MSE
std = np.std(MSE) #standard deviation
print(mean, std)

150.6537707617229 0.0


- Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.
- Report the mean and the standard deviation of the mean squared errors.

In [19]:
n_mse=50 #number of mean squared errors
MSEs=[] #empty array to store 50 mean squared errors

# Intializing loop to create a list of 50 MSE
for k in range(0,n_mse):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=k) #training the data
    model.fit(X_train,y_train,epochs=100,verbose=0)
    MSE_eval=model.evaluate(X_test,y_test,verbose=0)
    
    print('The value of mean squared value',str(k+1),'is', str(MSE_eval))
    
    y_pred=model.predict(X_test)
    MSE=mean_squared_error(y_test, y_pred) #mean squared error
    MSEs.append(MSE)
    
MSEs=np.array(MSEs)
mean=np.mean(MSEs)
std=np.std(MSEs)

print('The mean of mean squared errors is: ',mean)
print('The standard deviation of mean squared errors is: ',std)

The value of mean squared value 1 is 85.3145910590212
The value of mean squared value 2 is 76.35988735458226
The value of mean squared value 3 is 56.15709509741527
The value of mean squared value 4 is 45.86943585047058
The value of mean squared value 5 is 42.66706339518229
The value of mean squared value 6 is 43.75401303683284
The value of mean squared value 7 is 45.90804629032666
The value of mean squared value 8 is 34.043435920789406
The value of mean squared value 9 is 37.65031092375227
The value of mean squared value 10 is 38.644225605097404
The value of mean squared value 11 is 38.346386338514804
The value of mean squared value 12 is 33.664422167929246
The value of mean squared value 13 is 42.632586334129755
The value of mean squared value 14 is 42.96981726340877
The value of mean squared value 15 is 35.17348256774705
The value of mean squared value 16 is 32.86754405074135
The value of mean squared value 17 is 38.4318136196692
The value of mean squared value 18 is 35.8768595167734

Part C: Baseline model is built with normalized data using Keras library with epochs = 100.

The mean of mean squared errors using normalized data with epochs = 100 is further decreased as compared to normalized data with epochs = 50. This means less error between predicted data and validation test data using higher number of iterations (epochs).